In [1]:
import cv2
import numpy as np
import random


In [2]:
# 1. Horizontal Flip
def horizontal_flip(image):
    return cv2.flip(image, 1)

In [3]:
# 2. Vertical Flip
def vertical_flip(image):
    return cv2.flip(image, 0)

In [4]:
# 3. Rotation
def rotate_image(image, angle=45):
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    return cv2.warpAffine(image, M, (w, h))

In [5]:
# 4. Random Crop
def random_crop(image, crop_size=(100,100)):
    h, w = image.shape[:2]
    ch, cw = crop_size
    if ch > h or cw > w:   # prevent crop errors
        return image
    x = random.randint(0, w - cw)
    y = random.randint(0, h - ch)
    return image[y:y+ch, x:x+cw]


In [6]:
# 5. Brightness Adjustment
def adjust_brightness(image, factor=1.5):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hsv[:,:,2] = np.clip(hsv[:,:,2] * factor, 0, 255)
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

In [7]:
# 6. Gaussian Noise
def add_gaussian_noise(image, mean=0, sigma=25):
    gauss = np.random.normal(mean, sigma, image.shape).astype('uint8')
    noisy = cv2.add(image, gauss)
    return noisy

In [8]:
# 7. Gaussian Blur
def gaussian_blur(image, ksize=(5,5)):
    return cv2.GaussianBlur(image, ksize, 0)

In [9]:
# 8. Scaling (Zoom)
def scale_image(image, scale=1.5):
    h, w = image.shape[:2]
    return cv2.resize(image, (int(w*scale), int(h*scale)))

In [10]:
# 9. Shearing
def shear_image(image, shear_factor=0.2):
    h, w = image.shape[:2]
    M = np.array([[1, shear_factor, 0],
                  [0, 1, 0]], dtype=float)
    return cv2.warpAffine(image, M, (w, h))

In [11]:
# 10. Grayscale
def to_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [14]:
def flip_both(image):
    return cv2.flip(image, -1)  # -1 flips both axes


In [15]:
def random_rotation(image, max_angle=30):
    angle = random.uniform(-max_angle, max_angle)
    return rotate_image(image, angle)


In [16]:
def random_brightness_contrast(image):
    alpha = random.uniform(0.8, 1.2)  # contrast
    beta = random.randint(-30, 30)    # brightness
    return cv2.convertScaleAbs(image, alpha=alpha, beta=beta)


In [17]:
def random_perspective(image, scale=0.1):
    h, w = image.shape[:2]
    pts1 = np.float32([[0,0],[w,0],[0,h],[w,h]])
    delta = scale * min(h,w)
    pts2 = pts1 + np.random.uniform(-delta, delta, pts1.shape).astype(np.float32)
    M = cv2.getPerspectiveTransform(pts1, pts2)
    return cv2.warpPerspective(image, M, (w, h))


In [18]:
def motion_blur(image, kernel_size=15):
    kernel = np.zeros((kernel_size, kernel_size))
    kernel[int((kernel_size-1)/2), :] = np.ones(kernel_size)
    kernel /= kernel_size
    return cv2.filter2D(image, -1, kernel)


In [19]:
def salt_and_pepper(image, amount=0.02):
    output = np.copy(image)
    num_salt = np.ceil(amount * image.size * 0.5)
    num_pepper = np.ceil(amount * image.size * 0.5)
    
    # Salt
    coords = [np.random.randint(0, i - 1, int(num_salt)) for i in image.shape[:2]]
    output[coords[0], coords[1], :] = 255
    
    # Pepper
    coords = [np.random.randint(0, i - 1, int(num_pepper)) for i in image.shape[:2]]
    output[coords[0], coords[1], :] = 0
    
    return output


In [20]:
def channel_shift(image, shift_limit=50):
    image = image.astype(np.int32)
    for c in range(3):
        image[:,:,c] += random.randint(-shift_limit, shift_limit)
    return np.clip(image, 0, 255).astype(np.uint8)


In [21]:
def cutout(image, size=50):
    h, w = image.shape[:2]
    x = random.randint(0, w - size)
    y = random.randint(0, h - size)
    image[y:y+size, x:x+size] = 0
    return image


In [26]:
from scipy.ndimage import map_coordinates, gaussian_filter

def elastic_transform(image, alpha=34, sigma=4):
    random_state = np.random.RandomState(None)
    shape = image.shape[:2]

    dx = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma) * alpha
    dy = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma) * alpha
    x, y = np.meshgrid(np.arange(shape[1]), np.arange(shape[0]))
    indices = (np.reshape(y + dy, (-1,)),
               np.reshape(x + dx, (-1,)))

    # Apply transform separately to each channel
    transformed = np.zeros_like(image)
    for i in range(image.shape[2]):
        channel = image[..., i]
        distorted = map_coordinates(channel, indices, order=1, mode='reflect')
        transformed[..., i] = distorted.reshape(shape)

    return transformed


In [23]:
def random_hue_saturation(image, hue_shift=10, sat_shift=30):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV).astype(np.int32)
    hsv[:,:,0] = (hsv[:,:,0] + random.randint(-hue_shift, hue_shift)) % 180
    hsv[:,:,1] = np.clip(hsv[:,:,1] + random.randint(-sat_shift, sat_shift), 0, 255)
    return cv2.cvtColor(hsv.astype(np.uint8), cv2.COLOR_HSV2BGR)


In [24]:
# ==============================
# Master function to apply all 20 augmentations
# ==============================
def augment_all(image_path):
    image = cv2.imread(image_path)

    augmentations = {
        # Original 10
        "horizontal_flip": horizontal_flip(image),
        "vertical_flip": vertical_flip(image),
        "rotation": rotate_image(image, 45),
        "random_crop": random_crop(image, (150, 150)),
        "brightness": adjust_brightness(image, 1.5),
        "gaussian_noise": add_gaussian_noise(image),
        "gaussian_blur": gaussian_blur(image),
        "scaled": scale_image(image, 1.2),
        "sheared": shear_image(image),
        "grayscale": to_grayscale(image),

        # New 10
        "flip_both": flip_both(image),
        "random_rotation": random_rotation(image),
        "random_brightness_contrast": random_brightness_contrast(image),
        "random_perspective": random_perspective(image),
        "motion_blur": motion_blur(image),
        "salt_and_pepper": salt_and_pepper(image),
        "channel_shift": channel_shift(image),
        "cutout": cutout(image.copy()),  # copy to avoid overwriting
        "elastic_transform": elastic_transform(image),
        "random_hue_saturation": random_hue_saturation(image)
    }

    # Save augmented images
    results = {}
    for name, aug_img in augmentations.items():
        out_path = f"output_{name}.jpg"
        cv2.imwrite(out_path, aug_img)
        results[name] = out_path

    return results


In [27]:
# ==============================
# Run example
# ==============================
if __name__ == "__main__":
    path = "DIvine.jpg"   # replace with uploaded image
    results = augment_all(path)
    print("✅ Augmented images saved successfully!")

✅ Augmented images saved successfully!
